## Exercise 26

In [1]:
import mysql.connector
from mysql.connector import Error

### Class: Database
### Description: A class for managing database operations.

In [2]:
class Database:
    def __init__(self, host, user, password, database=None):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.connection = None
        self.cursor = None
        self.connect()
 
    
    def connect(self):
        try:
            self.connection = mysql.connector.connect(
                host=self.host,
                user=self.user,
                password=self.password,
                database=self.database
            )
            self.cursor = self.connection.cursor()
        except Error as e:
            print(e)


    def create_database(self, name):
        try:
            sql = f"CREATE DATABASE IF NOT EXISTS {name}"
            self.cursor.execute(sql)
            self.connection.database = name
            self.database = name
            print("Database Created!")
        except Error as e:
            print(e)


    def create_table(self, table_sql):
        try:
            self.cursor.execute(table_sql)
            print(f"{table_sql}: created")
        except Error as e:
            print(e)


    def execute_query(self, query, params=None):
        try:
            self.cursor.execute(query, params or ())
            if self.cursor.with_rows:
                return self.cursor.fetchall()
        except Error as e:
            print(e)


    def commit(self):
        self.connection.commit()

   
    def close(self):
        self.connection.close()

### Class: User
### Description: A class for managing user operations.

In [3]:
class User:
    def __init__(self, db):
        self.db = db

    
    def register_member(self, name, email, phone, username, password):
        ## Checking uniqueness of username:
        query = "SELECT username from users WHERE username = %s"
        result = self.db.execute_query(query, (username,))
        if not result:
            query = "INSERT INTO users (name, email, phone, username, password) VALUES (%s, %s, %s, %s, %s)"
            self.db.execute_query(query, (name, email, phone, username, password))
            self.db.commit()
        else:
            print(f"The provided username `{username}` is already exists.")


    def member_login(self, username, password):
        query = "SELECT * from users WHERE username = %s and password = %s"
        result = self.db.execute_query(query, (username, password))
        if result:
            print("The login was successful.")
        else:
            print("The provided username or password is invalid.")


    def show_profile_details(self, username):
        query = "SELECT name, email, phone FROM users where username = %s"
        result = self.db.execute_query(query, (username,))
        if result:
            print("Name      Email         Phone\n" + "-"*40)
            for field in result[0]:
                print(field, end="\t")
        else:
            print(f"No user was found for the provided username '{username}'.")


### Class: Employee
### Description: A class for managing employee operations.

In [4]:
class Employee:
    def __init__(self, db):
        self.db = db

    def add_employee(self, name, email, phone, position, username, password):
        ## Checking uniqueness of username:
        query = "SELECT username from employees WHERE username = %s"
        result = self.db.execute_query(query, (username,))
        if not result:
            query = "INSERT INTO employees (name, email, phone, position, username, password) VALUES (%s, %s, %s, %s, %s, %s)"
            self.db.execute_query(query, (name, email, phone, position, username, password))
            self.db.commit()
        else:
            print(f"The provided username `{username}` is already exists.")


    def show_employee_details(self, username):
        query = "SELECT name, email, phone, position FROM employees where username = %s"
        result = self.db.execute_query(query, (username,))
        if result:
            print("Name      Email         Phone        Position\n" + "-"*50)
            for field in result[0]:
                print(field, end="\t")
        else:
            print(f"No employee was found for the provided username '{username}'.")
        

### Class: Book
### Description: A class for managing book operations.

In [5]:
class Book:
    def __init__(self, db):
        self.db = db
        

    def add_book(self, title, author, genre, publication_year, edition):
        query = "INSERT INTO books (title, author, genre, publication_year, edition, availability) VALUES (%s, %s, %s, %s, %s, %s)"
        self.db.execute_query(query, (title, author, genre, publication_year, edition, True))
        self.db.commit()


    def update_book_info(self, **kwargs):
        ## Updating a book is based on its id, so the kwargs dictionary must inclue the `id` key. 
        ## the other items in kwargs are used for updating author, publication_year and genre
        book_id = kwargs.get('id')
        if not book_id: 
            print("The `kwargs` parameter must include an `id` argument that represents the book ID to be updated.")
            return
        ## removing `id` key from kwargs
        kwargs.pop('id')
        ## Iterating over the Python kwargs dictionary (remaining items are update options)
        q_update_options = []
        params = []
        for coloumn, new_val in kwargs.items():
            q_update_options.append(f"{coloumn} = %s")
            params.append(new_val)
        q_update_options_str = ", ".join(q_update_options)
        query = f"UPDATE books SET {q_update_options_str} WHERE id = {book_id}"
        self.db.execute_query(query, tuple(params))
            

    def search_books(self, **kwargs):
        ## search by title, author and genre
        ## Iterating over the Python kwargs dictionary
        q_conditions = []
        params = []
        for coloumn, s_term in kwargs.items():
            q_conditions.append(f"{coloumn} LIKE '%{s_term}%'")
        q_conditions_string = " AND ".join(q_conditions)
        query = f"SELECT * FROM books WHERE {q_conditions_string}" 
        result = self.db.execute_query(query)
        if result:
            print("Search Results:")
            for item in result:
                print(item)
        else:
            print("Nothing Found!")
            

### Defining tables:

In [6]:
tables = {
    'users': """
        CREATE TABLE IF NOT EXISTS users(
            id INT AUTO_INCREMENT,
            username VARCHAR(255) NOT NULL,
            password VARCHAR(255) NOT NULL,
            name VARCHAR(255) NOT NULL,
            email  VARCHAR(255) NOT NULL,
            phone VARCHAR(255) NOT NULL,
            PRIMARY KEY (id)
        );
    """,
    'employees': """
        CREATE TABLE IF NOT EXISTS employees(
            id INT AUTO_INCREMENT,
            username VARCHAR(255) NOT NULL,
            password VARCHAR(255) NOT NULL,
            name VARCHAR(255) NOT NULL,
            email  VARCHAR(255) NOT NULL,
            phone VARCHAR(255) NOT NULL,
            position VARCHAR(255) NOT NULL,
            PRIMARY KEY (id)
        );
    """,
    'books': """
        CREATE TABLE IF NOT EXISTS books(
            id INT AUTO_INCREMENT,
            title VARCHAR(255) NOT NULL,
            author VARCHAR(255) NOT NULL,
            genre VARCHAR(255) NOT NULL,
            publication_year YEAR,
            edition INT,
            availability BOOL NOT NULL,
            PRIMARY KEY (id)
        );
    """
}


### main part of the program

In [7]:
from configparser import ConfigParser 
## Retrieve the relevant information of the MySQL connection from a configuration file named `config.ini`
configur = ConfigParser() 
configur.read('config.ini')
host = configur['MySQL']['host']
user = configur['MySQL']['user']
password = configur['MySQL']['password']
database = configur['MySQL']['database']

db_info = {
    'host': host,
    'user': user,
    'password': password
}
db = Database(**db_info)
## Creating and Connecting to the Database:
db.create_database(database)


Database Created!


In [8]:
query = "SHOW TABLES"
print(db.execute_query(query))


[]


### Creating tables (users, employees and books)

In [9]:
for table in tables.values():
    db.create_table(table)
    


        CREATE TABLE IF NOT EXISTS users(
            id INT AUTO_INCREMENT,
            username VARCHAR(255) NOT NULL,
            password VARCHAR(255) NOT NULL,
            name VARCHAR(255) NOT NULL,
            email  VARCHAR(255) NOT NULL,
            phone VARCHAR(255) NOT NULL,
            PRIMARY KEY (id)
        );
    : created

        CREATE TABLE IF NOT EXISTS employees(
            id INT AUTO_INCREMENT,
            username VARCHAR(255) NOT NULL,
            password VARCHAR(255) NOT NULL,
            name VARCHAR(255) NOT NULL,
            email  VARCHAR(255) NOT NULL,
            phone VARCHAR(255) NOT NULL,
            position VARCHAR(255) NOT NULL,
            PRIMARY KEY (id)
        );
    : created

        CREATE TABLE IF NOT EXISTS books(
            id INT AUTO_INCREMENT,
            title VARCHAR(255) NOT NULL,
            author VARCHAR(255) NOT NULL,
            genre VARCHAR(255) NOT NULL,
            publication_year YEAR,
            edition INT,
    

In [10]:
query = "SHOW TABLES"
print(db.execute_query(query))


[('books',), ('employees',), ('users',)]


### Testing methods of the `User` class

In [11]:
user = User(db)


In [12]:
user.register_member('n1', 'e1@yahoo.com', '03142264556', 'username1', 'pass')
user.register_member('n2', 'e2@yahoo.com', '02142261234', 'username2', 'pass')
user.register_member('n3', 'e3@yahoo.com', '06142266789', 'username3', 'pass')
user.register_member('n5', 'e5@yahoo.com', '0615326543', 'username1', 'pass2') # Error


The provided username `username1` is already exists.


In [13]:
query = "SELECT * FROM users"
items = db.execute_query(query)
for item in items:
    print(item)
    

(1, 'username1', 'pass', 'n1', 'e1@yahoo.com', '03142264556')
(2, 'username2', 'pass', 'n2', 'e2@yahoo.com', '02142261234')
(3, 'username3', 'pass', 'n3', 'e3@yahoo.com', '06142266789')


In [15]:
user.member_login('username1', 'pass') # Successful
user.member_login('username1', 'pass1') # Faild
print("-"* 50)
user.show_profile_details(username= 'username10') # Not found
print("-"* 50)
user.show_profile_details(username= 'username3')


The login was successful.
The provided username or password is invalid.
--------------------------------------------------
No user was found for the provided username 'username10'.
--------------------------------------------------
Name      Email         Phone
----------------------------------------
n3	e3@yahoo.com	06142266789	

### Testing methods of the `Employee` class

In [16]:
emp = Employee(db)


In [17]:
emp.add_employee('n3', 'e3@gmail.com', '09345678901', 'Circulation Clerk', 'user1', 'pass')
emp.add_employee('n4', 'e4@gmail.com', '09123456789', 'Librarian', 'user2', 'pass')


In [18]:
query = "SELECT * FROM employees"
items = db.execute_query(query)
for item in items:
    print(item)
    

(1, 'user1', 'pass', 'n3', 'e3@gmail.com', '09345678901', 'Circulation Clerk')
(2, 'user2', 'pass', 'n4', 'e4@gmail.com', '09123456789', 'Librarian')


In [19]:
emp.show_employee_details('user0') # Not found
print("-"* 50)
emp.show_employee_details('user1')


No employee was found for the provided username 'user0'.
--------------------------------------------------
Name      Email         Phone        Position
--------------------------------------------------
n3	e3@gmail.com	09345678901	Circulation Clerk	

### Testing methods of the `Book` class

In [20]:
b_obj = Book(db)


In [21]:
## Testing `add_book` method of the `Book` class
b_obj.add_book("One Hundred Years of Solitude", "a1", "g1", '1967', 1)
b_obj.add_book("Animal Farm", "a2", "Allegory", '1930', 1)
b_obj.add_book("Love in the Time of Cholera", "Gabriel Garcia Marquez", "Romance Novel", '1985', 1)

query = "SELECT * FROM books"
items = db.execute_query(query)
for item in items:
    print(item)


(1, 'One Hundred Years of Solitude', 'a1', 'g1', 1967, 1, 1)
(2, 'Animal Farm', 'a2', 'Allegory', 1930, 1, 1)
(3, 'Love in the Time of Cholera', 'Gabriel Garcia Marquez', 'Romance Novel', 1985, 1, 1)


In [22]:
## Testing `update_book_info` method of the `Book` class
b_obj.update_book_info(id= 1, genre= "Magical Realism", author= "Gabriel Garcia Marquez")
b_obj.update_book_info(id= 2, publication_year= '1945', author= "George Orwell")
b_obj.update_book_info(id= 3, dummy_col= "value") # prints handled Error
b_obj.update_book_info(author= "Haroki Morakami") # prints handled Error: `id` is missing
print("-"* 50)
query = "SELECT * FROM books"
items = db.execute_query(query)
for item in items:
    print(item)
    

1054 (42S22): Unknown column 'dummy_col' in 'field list'
The `kwargs` parameter must include an `id` argument that represents the book ID to be updated.
--------------------------------------------------
(1, 'One Hundred Years of Solitude', 'Gabriel Garcia Marquez', 'Magical Realism', 1967, 1, 1)
(2, 'Animal Farm', 'George Orwell', 'Allegory', 1945, 1, 1)
(3, 'Love in the Time of Cholera', 'Gabriel Garcia Marquez', 'Romance Novel', 1985, 1, 1)


In [23]:
## Testing `search_books` method of the `Book` class
b_obj.search_books(author= 'Marquez')
print("-"* 50)
b_obj.search_books(title= 'of', author='Gabriel')
print("-"* 50)
b_obj.search_books(genre= 'Thriller') # Not found

Search Results:
(1, 'One Hundred Years of Solitude', 'Gabriel Garcia Marquez', 'Magical Realism', 1967, 1, 1)
(3, 'Love in the Time of Cholera', 'Gabriel Garcia Marquez', 'Romance Novel', 1985, 1, 1)
--------------------------------------------------
Search Results:
(1, 'One Hundred Years of Solitude', 'Gabriel Garcia Marquez', 'Magical Realism', 1967, 1, 1)
(3, 'Love in the Time of Cholera', 'Gabriel Garcia Marquez', 'Romance Novel', 1985, 1, 1)
--------------------------------------------------
Nothing Found!


### Closing the connection to database

In [23]:
db.close()